In [2]:
import pandas as pd
import numpy as np
import pydicom 
import os
import matplotlib.pyplot as plt

from keras.models import Model, Sequential
from keras.layers import Concatenate, Dense, Input, concatenate
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

Using TensorFlow backend.


In [3]:
def load_best_slices(path):
    ind = []
    for file in os.listdir(path):
        if file.split('.')[1] == 'txt':
            ind.append(file.split('.')[0])
        
    df = pd.DataFrame(index = ind, columns= ['CT'])
    for ind in df.index:
        df.loc[ind].CT = np.loadtxt(path + ind + '.txt')
    
    return df

df = load_best_slices('best_lung_slice/')

In [4]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.index, columns = ['slope', 'intersect'])

for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope
    result.loc[ind].intersect = linear_data_all.loc[ind].intersect

In [5]:
for ind in df.CT.index:
    if df.loc[ind].CT.shape != (512,512):
        df = df.drop(ind)

In [6]:
linear_data_all = pd.read_csv('patient_slope_intercept.csv', index_col=0)
result = pd.DataFrame(index = df.index, columns = ['slope', 'intersect'])

for ind in result.index:
    result.loc[ind].slope = linear_data_all.loc[ind].slope
    result.loc[ind].intersect = linear_data_all.loc[ind].intersect

In [7]:
dataset = df.values

In [8]:
X = []
for ind in df.index:
    X.append(df.loc[ind].CT)

X = np.array(X)


In [9]:
X = X.reshape(X.shape[0], 512, 512, 1)
y = result.values

In [9]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(2,2), strides=(1,1), padding='valid', activation='relu', input_shape=(512,512,1)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(20, activation='relu'))
# output layer
model.add(Dense(2, activation='sigmoid'))

In [10]:
model.compile(loss='mean_squared_error', metrics=['accuracy'], optimizer='adam')

In [1]:
model.fit(X,y, batch_size=10, epochs=100)

NameError: name 'model' is not defined